In [ ]:
!python -m pip install --upgrade pip
%pip install pandas matplotlib seaborn scikit-learn openpyxl tensorflow xgboost aif360
%pip install "aif360[Reductions, inFairness]"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.stats import chi2_contingency, fisher_exact
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, BatchNormalization # type: ignore

from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.algorithms.preprocessing import LFR
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.inprocessing import GerryFairClassifier, PrejudiceRemover, MetaFairClassifier
from aif360.algorithms.postprocessing import EqOddsPostprocessing, RejectOptionClassification

from fairlearn.metrics import MetricFrame
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference, selection_rate, false_positive_rate, false_negative_rate, count
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

random_seed = 15

pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [ ]:
PATH = 'C:/Users/aberti/Desktop/ProjectWork_AEQUITAS_AKKODIS/data/'
df = (
    pd.read_excel(PATH + 'Dataset_2.0_Akkodis.xlsx')
      .rename(columns=lambda c: c.lstrip().title())
)
df.head()

### Clean Dataset

In [ ]:
df = df.drop_duplicates(subset='Id', keep='last')
df.head()

In [ ]:
unuseful_columns = [
    'ID', 'TAG', 'Year of insertion', 'Year of Recruitment', 'Recruitment Request', 
    'Assumption Headquarters', 'event_type__val', 'linked_search__key', 'Job Description', 
    'Candidate Profile', 'Akkodis headquarters', 'Standing/Position', 'Unnamed: 0', 
    'Residence', 'Last Role', 'Study Area.1', 'Years Experience.1']
df = df.drop(columns=unuseful_columns)
df.head()

In [ ]:
THRESHOLD = 0.4
unuseful_columns = []
for col in df.columns:
  null_count = df[col].isna().sum() / df.shape[0]
  if null_count > THRESHOLD:
    unuseful_columns.append(col)
    print(f'DROPPED <Column: {col}> NULL count: {null_count*100:.2f}%')
  else:
    print(f'<Column: {col}> NULL count: {null_count*100:.2f}%')
  
df = df.drop(columns=unuseful_columns)
df.head()

In [ ]:
for feature in df.columns:
    print(f'Feature: {feature} -- {list(df[feature].unique())}')

In [ ]:
rows_mappings = {
    'Protected Category': {
        'Article 18': 'Yes',
        'Article 1': 'Yes'
    }
}
for col, mapping in rows_mappings.items():
    df[col] = df[col].replace(mapping)
df.head()

In [ ]:
# Remove rows with 'Candidate State' as 'First contact' or 'Imported'
# These statuses are not relevant for the analysis
rows_to_remove = {
    'Candidate State': ['First contact', 'Imported']
}

for col, remove_list in rows_to_remove.items():
    df = df[~df[col].isin(remove_list)]

df.head()

### Handle the NANs

In [ ]:
print(f'Columns that contain NaN values:\n {df.columns[df.isnull().any()].tolist()}')

for col in df.columns[df.isnull().any()].tolist():
  print(f'{col} values: {df[col].unique()} \n') # Analyze each NaN containing feature first to determine the default fill value

In [ ]:
fill_values = {
    'Citizenship': 'Not Specified',
    'Protected category': 'Not a protected category',
    'Study area': 'Not Specified',
    'Sector': 'Unemployed',
    'Job Family Hiring': 'Not Specified',
    'Job Title Hiring': 'Not Specified',
    'vent_feedback': 'Not Specified',
    'verall': 'Not Specified',
    'Minimum Ral': 'Not Specified',
    'Ral Maximum': 'Not Specified',
    'Study Level': 'Not Specified',
    'Current Ral': 'Not Specified',
    'Expected Ral': 'Not Specified',
    'Technical Skills': df['Technical Skills'].mean(),
    'Comunication': df['Comunication'].mean(),
    'Maturity': df['Maturity'].mean(),
    'Dynamism': df['Dynamism'].mean(),
    'Mobility': df['Mobility'].mean(),
    'English': df['English'].mean()
}
df = df.fillna(fill_values)
print(f'There are {df.isnull().sum().sum()} NANs')
df.head()

### Features Reformatting

In [ ]:
print(df['Residence'].unique())

In [ ]:
rename_keywords = [
    'ETHIOPIA',
    'SOUTH AFRICAN REPUBLIC',
    'USSR',
    'YUGOSLAVIA'
]

for kw in rename_keywords:
  mask = df['Residence'].str.contains(kw, na=False)
  df.loc[mask, 'Residence'] = f"{kw} » (STATE) ~ (FOREIGN)"
  print(f"Formatted '{kw}' as '{kw} » (STATE) ~ (FOREIGN)'")

In [ ]:
residence_list = df['Residence'].unique()
state_list = [s for s in residence_list 
              if ('(STATE)' in s) 
              or ('(COUNTRY)' in s)
              and not ('Not Specified' in s)
              ]
italy_list = [s for s in residence_list 
              if not ('(STATE)' in s) 
              and not ('(COUNTRY)' in s)
              and not ('Not Specified' in s)
              ]
state_list = sorted({s.split(' » ')[0] for s in state_list if ' » ' in s})
city_list = sorted({s.split(' » ')[0] for s in italy_list if ' » ' in s})
province_list = sorted({s.split(' » ')[1].split(' ~ ')[0] for s in italy_list if ' » ' in s and ' ~ ' in s})
region_list = sorted({s.split(' ~ ')[-1] for s in italy_list if ' ~ ' in s})

print(f"List of residence italian citys of the candidates in the dataset:\n {city_list}")
print(f"List of residence italian provinces of the candidates in the dataset:\n {province_list}")
print(f"List of residence italian regions of the candidates in the dataset:\n {region_list}")
print(f"List of residence states of the candidates in the dataset:\n {state_list}")

def map_residence_city(value):
    for city in city_list:
        if city in value:
            return city
    return 'Not Specified'

def map_residence_province(value):
    for prov in province_list:
        if prov in value:
            return prov
    return 'Not Specified'

def map_residence_region(value):
    for region in region_list:
        if region in value:
            return region
    return 'Not Specified'

def map_residence_state(value):
    for state in state_list:
        if state in value:
            return state
    return 'ITALY'

# Applicazione sulle colonne
df['Residence City']     = df['Residence'].apply(map_residence_city)
df['Residence Province'] = df['Residence'].apply(map_residence_province)
df['Residence Region']   = df['Residence'].apply(map_residence_region)
df['Residence State']    = df['Residence'].apply(map_residence_state)
df.head()


In [ ]:
european_countries = [
    'ALBANIA', 'AUSTRIA', 'BELARUS', 'BELGIUM', 'BULGARIA', 'CROATIA', 'CZECH REPUBLIC',
    'FRANCE', 'GERMANY', 'GREAT BRITAIN-NORTHERN IRELAND', 'GREECE', 'ITALY', 'LATVIA',
    'LITHUANIA', 'LUXEMBOURG', 'MALTA', 'MOLDOVA', 'MONACO', 'MONTENEGRO', 'NETHERLANDS',
    'NORWAY', 'POLAND', 'PORTUGAL', 'ROMANIA', 'RUSSIA', 'SAN MARINO', 'SERBIA', 'SLOVAKIA',
    'SLOVENIA', 'SPAIN', 'SWEDEN', 'SWITZERLAND', 'UKRAINE'
]
df['European Residence'] = df['Residence State'].apply(lambda x: 'Yes' if x in european_countries else 'No')
df['Italian Residence'] = df['Residence State'].apply(lambda x: 'Yes' if 'ITALY' in x else 'No')
df.head()

### Features Remapping

In [ ]:
feature_mapping = {
    'Study area' : {
        'Automation/Mechatronics Engineering': 'Engineering',
        'computer engineering': 'Engineering',
        'chemical engineering': 'Engineering',
        'Legal': 'Law',
        'Mechanical engineering': 'Engineering',
        'Telecommunications Engineering': 'Engineering',
        'Economic - Statistics': 'Economic',
        'Psychology': 'Scientific Field',
        'Materials Science and Engineering': 'Engineering',
        'Other scientific subjects': 'Scientific Field',
        'Biomedical Engineering': 'Engineering',
        'electronic Engineering': 'Engineering',
        'Information Engineering': 'Engineering',
        'Aeronautical/Aerospace/Astronautics Engineering': 'Engineering',
        'Energy and Nuclear Engineering': 'Engineering',
        'Informatics': 'Informatics',
        'Management Engineering': 'Engineering',
        'Automotive Engineering': 'Engineering',
        'industrial engineering': 'Engineering',
        'Other': 'Other',
        'Surveyor': 'NO COLLEGE',
        'Civil/Civil and Environmental Engineering': 'Engineering',
        'Electrical Engineering': 'Engineering',
        'Scientific maturity': 'NO COLLEGE',
        'Chemist - Pharmaceutical': 'Medical Field',
        'Political-Social': 'Other Humanities Subjects',
        'Other humanities subjects': 'Other Humanities Subjects',
        'Geo-Biological': 'Scientific Field',
        'Linguistics': 'Linguistics',
        'Agriculture and veterinary': 'Scientific Field',
        'Literary': 'Other Humanities Subjects',
        'Humanistic high school diploma': 'NO COLLEGE',
        'Accounting': 'NO COLLEGE',
        'Communication Sciences': 'Other Humanities Subjects',
        'Safety Engineering': 'Engineering',
        'Architecture': 'Scientific Field',
        'Mathematics': 'Scientific Field',
        'construction Engineering': 'Engineering',
        'Petroleum Engineering': 'Engineering',
        'Naval Engineering': 'Engineering',
        'Artistic': 'NO COLLEGE',
        'Not Specified': 'Other',
        'Mathematical-physical modeling for engineering': 'Engineering',
        'Engineering for the environment and the territory': 'Engineering',
        'Medical': 'Medical Field',
        'Defense and Security': 'Other',
        'Physical education': 'Other',
        'Statistics': 'Scientific Field',
        'Educational/training sciences': 'Other Humanities Subjects'
    },
    'Age Range' : {
        '< 20 years': '< 20 years',
        '20 - 25 years': '20 - 25 years',
        '26 - 30 years': '26 - 30 years',
        '31 - 35 years': '31 - 35 years',
        '36 - 40 years': '36 - 40 years',
        '40 - 45 years': '40 - 45 years',
        '> 45 years': '> 45 years'
    }
}
for col, mapping in feature_mapping.items():
    df[col] = df[col].replace(mapping)
df.head()

### Target Column

In [ ]:
status_positive_conditions = {
    'Candidate State': ['Hired', 'Economic proposal', 'QM'],
    'event_feedback': ['OK (live)', 'OK (waiting for departure)', 'OK (hired)']
}

mask = np.zeros(len(df), dtype=bool)
for col, valid_values in status_positive_conditions.items():
    mask |= df[col].isin(valid_values)

df['Status'] = np.where(mask, 'Positive', 'Negative')

### Categorical columns

In [ ]:
encoding_mappings = {}

categorical_columns = [
    'Status', 'Age Range', 'Citizenship', 'Sex',
    'Protected category', 'Study area', 'Study Title',
    'Years Experience', 'Sector', 'Job Family Hiring',
    'Job Title Hiring', 'Overall',
    'Minimum Ral', 'Ral Maximum', 'Study Level',
    'Current Ral', 'Expected Ral'
]

custom_orders = {
    'Status': ['Negative', 'Positive'],
    'Age Range': ['< 20 years', '20 - 25 years', '26 - 30 years',
                  '31 - 35 years', '36 - 40 years', '40 - 45 years', '> 45 years'],
    'Years Experience': ['[0]', '[0-1]', '[1-3]', '[3-5]', '[5-7]', '[7-10]', '[+10]'],
}

for col in categorical_columns:
    if col in custom_orders:
        df[f"{col}_encoded"] = pd.Categorical(df[col], categories=custom_orders[col], ordered=True).codes
        encoding_mappings[col] = {cat: i for i, cat in enumerate(custom_orders[col])}
    else:
        encoder = LabelEncoder()
        df[f"{col}_encoded"] = encoder.fit_transform(df[col].astype(str))
        encoding_mappings[col] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

df = df.drop(columns=categorical_columns)
df.head()

### Visualize Data

In [ ]:
lookups = ['Citizenship', 'Age Range', 'Sex', 'Protected category', 'Study area', 'Study Title', 'Years Experience']
for lookup in lookups:
    mapping = {v: k for k, v in encoding_mappings[lookup].items()}

    distrib = Counter(df[f"{lookup}_encoded"])
    distrib_df = pd.DataFrame(distrib.items(), columns=[f"{lookup}_encoded", 'Count'])
    distrib_df[lookup] = distrib_df[f"{lookup}_encoded"].map(mapping)
    distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
    plt.title(lookup)
    plt.xticks(rotation=45)

#### Correlation Matrix

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0, linewidths=.5)

### Visualize Percentage of Hired inside each class

In [ ]:
sensitive = ['Sex_encoded', 'Age Range_encoded', 'Citizenship_encoded', 'Protected Category_encoded']
for feature in sensitive:
    percentage = df.groupby(feature)['STATUS'].mean().mul(100).round(2)
    for category, perc in percentage.items():
        print(f"Percentage of elements where {feature} is {category} and STATUS is HIRED: {percentage:.2f}%")

### Chi-squared Tests

In [ ]:
# Tabelle di contingenza
contingency_sex    = pd.crosstab(df['Sex'], df['Status'])
contingency_age    = pd.crosstab(df['Age Range'], df['Status'])
contingency_region = pd.crosstab(df['Residence Region'], df['Status'])

# Chi-squared tests
tables = {
    'Sex': contingency_sex,
    'Age Range': contingency_age,
    'Residence Region': contingency_region
}
for var, table in tables.items():
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    test_name = 'Chi-squared'
    
    # se 2×2 e attese <5 → Fisher’s exact
    if table.shape == (2,2) and (expected < 5).any():
        _, p = fisher_exact(table)
        test_name = "Fisher's exact"
    
    n = table.values.sum()
    k = min(table.shape)
    cramer_v = np.sqrt(chi2 / (n * (k-1)))
    
    # Stampa a video
    print(f"--- {var} ---")
    print("Expected frequencies:")
    print(pd.DataFrame(expected, index=table.index, columns=table.columns))
    print()
    print(f"{test_name}: χ² = {chi2:.2f}, p = {p:.3f}, dof = {dof}, Cramér’s V = {cramer_v:.3f}")
    print("Conclusion: Significant association between two variables (Dependent)" if p < 0.05 else "Conclusion: No significant association between two variables (Independent)")
    print()
    

In [ ]:
selection_order = [
    'Imported', 'In selection', 'First contact',
    'QM', 'Vivier', 'Economic proposal', 'Hired'
]
lookouts = ['Sex', 'Age Range', 'Residence Region']
contingency_tables = {}

for lookout in lookouts:
    for i, state in enumerate(selection_order):
        post_states = selection_order[i+1:]
        if post_states:
            df_state      = df[df['Candidate State'] == state]
            df_post_state = df[df['Candidate State'].isin(post_states)]
            contingency_tables[state] = pd.DataFrame({
                f'Post {state}': df_post_state.groupby(lookout, observed=True).size(), state: df_state.groupby(lookout, observed=True).size()
            }).fillna(0).astype(int)

    for var, table in contingency_tables.items():
        chi2, p, dof, expected = chi2_contingency(table, correction=False)
        test_name = 'Chi-squared'
        
        # se 2×2 e attese <5 → Fisher’s exact
        if table.shape == (2,2) and (expected < 5).any():
            _, p = fisher_exact(table)
            test_name = "Fisher's exact"
        
        n = table.values.sum()
        k = min(table.shape)
        cramer_v = np.sqrt(chi2 / (n * (k-1)))
        
        # Stampa a video
        print(f"--- {var} ---")
        print(f"Contingency table (by {lookout}):")
        print(table)
        print()
        print("Expected frequencies:")
        print(pd.DataFrame(expected, index=table.index, columns=table.columns))
        print()
        print(f"{test_name}: χ² = {chi2:.2f}, p = {p:.3f}, dof = {dof}, Cramér’s V = {cramer_v:.3f}")
        print("Conclusion: Significant association between two variables (Dependent)" if p < 0.05 else "Conclusion: No significant association between two variables (Independent)")
        print()

## Train

### Dataset Preparation

In [ ]:
df = shuffle(df, random_state=random_seed)
X_full = df.drop(columns=['STATUS'])
y = df['STATUS']
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y, test_size=0.2, random_state=random_seed, stratify=y)

X = df.drop(columns=['STATUS', 'sex'])
y = df['STATUS']
s = df['sex']
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(X, y, s, test_size=0.2, random_state=random_seed, stratify=y)

In [ ]:
train_df = X_train.copy()
train_df['target'] = y_train.values
train_df['sex'] = s_train.values

test_df = X_test.copy()
test_df['target'] = y_test.values
test_df['sex'] = s_test.values

train_ds = StandardDataset(
    train_df,
    label_name='target',
    favorable_classes=[1],
    protected_attribute_names=['sex'],
    privileged_classes=[[1]]
)

test_ds = StandardDataset(
    test_df,
    label_name='target',
    favorable_classes=[1],
    protected_attribute_names=['sex'],
    privileged_classes=[[1]]
)

In [ ]:
lfr = LFR(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}], k=10, Ax=5, Ay=5, Az=10, max_iter=50, verbose=1)

lfr = lfr.fit(train_ds)
X_train_lfr = lfr.transform(train_ds)
X_train_lfr_df = pd.DataFrame(X_train_lfr.features, columns=train_ds.feature_names)


clf = LogisticRegression(solver='liblinear')
clf.fit(X_train_lfr_df, y_train)
preds_lfr = clf.predict(pd.DataFrame(lfr.transform(test_ds).features, columns=train_ds.feature_names))

In [ ]:
cr = CorrelationRemover(sensitive_feature_ids=[train_ds.feature_names.index('sex')])

X_train_cr = cr.fit_transform(X_train)
X_train_cr_df = pd.DataFrame(X_train_cr.features, columns=train_ds.feature_names)

X_test_cr  = cr.transform(X_test)
X_test_cr_df = pd.DataFrame(X_test_cr.features, columns=train_ds.feature_names)

In [ ]:
gfc = GerryFairClassifier(
    unprivileged_groups=[{'sex': 0}],
    privileged_groups=[{'sex': 1}],
    estimator=LogisticRegression(solver='liblinear'),
    constraints='demographic_parity'
)
gfc.fit(train_ds)
pred_gfc = gfc.predict(test_ds)

In [ ]:
pr = PrejudiceRemover(
    sensitive_attr='sex',
    eta=25.0
)
pr.fit(train_ds)
pred_pr = pr.predict(test_ds)

In [ ]:
mfc = MetaFairClassifier(
    unprivileged_groups=[{'sex': 0}],
    privileged_groups=[{'sex': 1}],
    sensitive_attr='sex',
    tau=0.8
)
mfc.fit(train_ds)
pred_mfc = mfc.predict(test_ds)

In [ ]:
eop = EqOddsPostprocessing(
    unprivileged_groups=[{'sex': 0}],
    privileged_groups=[{'sex': 1}]
)
eop = eop.fit(train_ds, gfc.predict(train_ds))
pred_eop = eop.predict(test_ds)

In [ ]:
roc = RejectOptionClassification(
    unprivileged_groups=[{'sex': 0}],
    privileged_groups=[{'sex': 1}],
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    metric_name='Average odds difference'
)
roc = roc.fit(train_ds, gfc.predict(train_ds))
pred_roc = roc.predict(test_ds)

In [ ]:
def compute_fairness_metrics(y_true, y_pred, sensitive_features, label=None):
    mf = MetricFrame(
        metrics={
            'selection_rate': selection_rate,
            'dp_diff': demographic_parity_difference,
            'eo_diff': equalized_odds_difference,
            'fpr': false_positive_rate,
            'fnr': false_negative_rate,
            'count': count
        },
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    if label:
        print(f"=== {label} ===")
    print(mf.by_group)
    print("Overall:", mf.overall, "\n")
    return mf

In [ ]:
# Pre-Processing
compute_fairness_metrics(y_test, preds_lfr, s_test, label="LFR + LogisticRegression")

# In-processing
compute_fairness_metrics(y_test, pred_gfc.ravel(), s_test, label="GerryFairClassifier")
compute_fairness_metrics(y_test, pred_pr.ravel(), s_test, label="PrejudiceRemover")
compute_fairness_metrics(y_test, pred_mfc.ravel(), s_test, label="MetaFairClassifier")

# Post-processing
compute_fairness_metrics(y_test, pred_eop.ravel(), s_test, label="EqOddsPostprocessing")
compute_fairness_metrics(y_test, pred_roc.ravel(), s_test, label="RejectOptionClassification")

### Models

In [ ]:
def create_model(seed):
    tf.random.set_seed(seed)
    model = Sequential()
    model.add(Dense(128, input_dim=22, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(),
    'KNN': KNeighborsClassifier(),
    'Neural Network': create_model(random_seed)
}

In [ ]:
metrics = []
predictions = {}

for name, model in models.items():
    if name in ['Linear Regression', 'Decision Tree', 'Naive Bayes', 'XGBoost', 'KNN']:
        model.fit(X_train_full, y_train_full)
    elif name in ['Neural Network']:
        model.fit(X_train_full, y_train_full, epochs=15, batch_size=64, validation_split=0.2)
    else:
        print("Error in Models!"); break

    y_pred = model.predict(X_test_full)

    if name in ['Linear Regression', 'XGBoost', 'Neural Network']:
        y_pred = (y_pred > 0.5).astype(int)

    accuracy = round(accuracy_score(y_test_full, y_pred), 3)
    precision = round(precision_score(y_test_full, y_pred), 3)
    recall = round(recall_score(y_test_full, y_pred), 3)
    f1 = round(f1_score(y_test_full, y_pred), 3)
    roc_auc = round(roc_auc_score(y_test_full, y_pred), 3)

    metrics.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'ROC AUC': roc_auc
    })
    predictions[name] = y_pred

metrics = pd.DataFrame(metrics)
predictions_df = pd.DataFrame({
    'Linear Regression' : predictions['Linear Regression'],
    'Decision Tree' : predictions['Decision Tree'],
    'Naive Bayes' : predictions['Naive Bayes'],
    'XGBoost' : predictions['XGBoost'],
    'kNN' : predictions['KNN'],
    'Neural Network' : predictions['Neural Network']
})

## Fairness Metrics

#### **3.1 Demographic Parity**

In [ ]:
sensitive_features = [' Sex_encoded', ' Age Range_encoded', ' Citizenship_encoded', ' Protected category_encoded']
non_sensitive_features = ['Technical Skills', 'Comunication', 'Maturity', 'Dynamism', 'Mobility',
       'English', ' Study area_encoded', ' Study Title_encoded', ' Years Experience_encoded', ' Sector_encoded', ' Job Family Hiring_encoded',
       ' Job Title Hiring_encoded', ' Overall_encoded', ' Years Experience.1_encoded',' Minimum Ral_encoded', ' Ral Maximum_encoded',
       ' Study Level_encoded', 'Current Ral_encoded', 'Expected Ral_encoded']
models_list = [model for model in models]
tolerance = 0.15
significance_level = 0.1

In [ ]:
def calculate_demographic_parity(predictions, sensitive_attribute, name, significance_level, tolerance, activate_check=False):

    df = pd.DataFrame({
        'predictions': predictions,
        'sensitive_attribute': sensitive_attribute
    })
    prop = df.groupby('sensitive_attribute')['predictions'].mean()
    
    if activate_check:
        print(f"===\n{name}\n{prop}")

    if prop.shape[0] == 2:
        return 'T' if (prop.max() - prop.min()) <= tolerance else False
    else:
        contingency_table = pd.crosstab(df['predictions'], df['sensitive_attribute'])
        chi2, p, dof, expected = chi2_contingency(contingency_table)

        if activate_check and (expected < 5).any():
            print(f"Sparse contingency for {name}")
                
        return 'T' if p > significance_level else False


In [ ]:
table = []
for model in models:
    row = []
    for sensitive_feature in sensitive_features:
        result = calculate_demographic_parity(predictions[model], X_test_full[sensitive_feature], sensitive_feature, significance_level, tolerance, activate_check=True)
        row.append(result)
    table.append(row)
sf_df = pd.DataFrame(table, index = models_list, columns=sensitive_features)

#### **3.2 Equalized odds**

In [ ]:
def calculate_equalized_odds(predictions, true_labels, sensitive_attribute, name, tolerance, activate_check=False):
    df = pd.DataFrame({
        'predictions': predictions,
        'true_labels': true_labels,
        'sensitive_attribute': sensitive_attribute
    })
    tprs, fprs = [], []
    for _, group_df in df.groupby('sens'):
        tn, fp, fn, tp = confusion_matrix(group_df['true_labels'], group_df['predictions'], labels=[0, 1]).ravel()
        tprs.append(tp / (tp + fn) if tp + fn != 0 else 0)
        fprs.append(fp / (fp + tn) if fp + tn != 0 else 0)

    max_tpr_diff = max(tprs) - min(tprs)
    max_fpr_diff = max(fprs) - min(fprs)

    if activate_check:
            print(f"===\n{name}\nMax FPR diff: {max_fpr_diff}\nMax TPR diff: {max_tpr_diff}")

    return 'T' if (max_tpr_diff <= 2 * tolerance and max_fpr_diff <= 2 * tolerance) else False

In [ ]:
table = []
for model in models:
    row = []
    for sensitive_feature in sensitive_features:
        result = calculate_equalized_odds(predictions[model], y_test_full, X_test_full[sensitive_feature], sensitive_feature, tolerance, activate_check=False)
        row.append(result)
    table.append(row)
sf_df = pd.DataFrame(table, index = models_list, columns=sensitive_features)